In [6]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.primitives import serialization
import time
from datetime import datetime

# Generate RSA keys
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

# Sender (Alice): Create message with timestamp in UTC and sign it
def sign_message_with_timestamp(message):
    timestamp = int(time.time())  # Current timestamp (seconds since epoch)
    utc_timestamp = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')  # UTC format
    message_with_timestamp = f"{message}|Timestamp:{utc_timestamp}|Epoch:{timestamp}"
    signature = private_key.sign(
        message_with_timestamp.encode(),
        padding.PSS(mgf=padding.MGF1(SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        SHA256()
    )
    return message_with_timestamp, signature

# Receiver (Bob): Verify message, signature, and timestamp
def verify_message_with_timestamp(message_with_timestamp, signature, time_window=30):
    try:
        # Parse message and extract UTC and epoch timestamps
        parts = message_with_timestamp.split('|')
        message = parts[0]
        utc_timestamp = parts[1].split(':', 1)[1]
        epoch_timestamp = int(parts[2].split(':', 1)[1])
    except ValueError:
        return False, "Malformed message format."

    # Check timestamp freshness
    current_time = int(time.time())
    if current_time - epoch_timestamp > time_window:
        return False, f"Replay detected: Timestamp expired. UTC: {utc_timestamp}"

    # Verify signature
    try:
        public_key.verify(
            signature,
            message_with_timestamp.encode(),
            padding.PSS(mgf=padding.MGF1(SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            SHA256()
        )
        return True, "Message verified successfully."
    except Exception:
        return False, "Signature verification failed."

# Example Usage
message = "Transfer $1000 to Oscar"
message_with_timestamp, signature = sign_message_with_timestamp(message)

# First attempt (valid)
print("Message with timestamp:", message_with_timestamp)
result, explanation = verify_message_with_timestamp(message_with_timestamp, signature)
print("First attempt:", result, explanation)

# Replay attempt after time window expires
time.sleep(31)  # Simulate delay
result, explanation = verify_message_with_timestamp(message_with_timestamp, signature)
print("Replay attempt after time window:", result, explanation)

Message with timestamp: Transfer $1000 to Oscar|Timestamp:2024-11-18 21:22:39|Epoch:1731964959
First attempt: True Message verified successfully.
Replay attempt after time window: False Replay detected: Timestamp expired. UTC: 2024-11-18 21:22:39


In [7]:
import hmac
import hashlib
import random

# Shared secret key between Alice and Bob
shared_key = b'secret_key'

# Sender (Alice): Create message with nonce and MAC
def create_message_with_nonce(message):
    nonce = random.randint(100000, 999999)  # Generate random nonce
    print(f"Generated nonce (sample): {nonce}")  # Display sample nonce
    message_with_nonce = f"{message}|Nonce:{nonce}"
    mac = hmac.new(shared_key, message_with_nonce.encode(), hashlib.sha256).hexdigest()
    return message_with_nonce, mac

# Receiver (Bob): Verify message, MAC, and nonce
received_nonces = set()  # To track previously received nonces

def verify_message_with_nonce(message_with_nonce, mac):
    try:
        message, nonce_str = message_with_nonce.rsplit('|', 1)
        nonce = int(nonce_str.split(':')[1])
    except ValueError:
        return False, "Malformed message format."

    # Check nonce uniqueness
    if nonce in received_nonces:
        return False, "Replay detected: Nonce repeated."
    received_nonces.add(nonce)

    # Verify MAC
    expected_mac = hmac.new(shared_key, message_with_nonce.encode(), hashlib.sha256).hexdigest()
    if hmac.compare_digest(expected_mac, mac):
        return True, "Message verified successfully."
    else:
        return False, "MAC verification failed."

# Example Usage
message = "Transfer $1000 to Oscar"

# First attempt: Valid message
message_with_nonce, mac = create_message_with_nonce(message)
print("Message with nonce and MAC:", message_with_nonce, mac)
result, explanation = verify_message_with_nonce(message_with_nonce, mac)
print("First attempt:", result, explanation)

# Replay attempt: Invalid due to repeated nonce
result, explanation = verify_message_with_nonce(message_with_nonce, mac)
print("Replay attempt:", result, explanation)

Generated nonce (sample): 222949
Message with nonce and MAC: Transfer $1000 to Oscar|Nonce:222949 be68b6b7f61b743d11429128f702882e043c8a23784119ad90c149bf9ae116cb
First attempt: True Message verified successfully.
Replay attempt: False Replay detected: Nonce repeated.
